## first base line

In [4]:
import pandas as pd
import numpy as np

In [107]:
# 데이터 확인
train_features = pd.read_csv("train_features.csv")
train_target = pd.read_csv("train_target.csv")
test_features = pd.read_csv("test_features.csv")

In [108]:
train_features.head()

,id,Time,S1,S2,S3,S4
0,0,0.000000,0.0,0.0,0.0,0.0
1,0,0.000004,0.0,0.0,0.0,0.0
2,0,0.000008,0.0,0.0,0.0,0.0
3,0,0.000012,0.0,0.0,0.0,0.0
4,0,0.000016,0.0,0.0,0.0,0.0


In [109]:
train_target.head()

,id,X,Y,M,V
0,0,0.0,-400.0,50.0,0.4
1,1,400.0,0.0,100.0,1.0
2,2,-300.0,-200.0,25.0,0.4
3,3,200.0,-100.0,150.0,0.4
4,4,-300.0,-100.0,150.0,0.4


In [110]:
test_features.head()

,id,Time,S1,S2,S3,S4
0,2800,0.000000,0.0,0.0,0.0,0.0
1,2800,0.000004,0.0,0.0,0.0,0.0
2,2800,0.000008,0.0,0.0,0.0,0.0
3,2800,0.000012,0.0,0.0,0.0,0.0
4,2800,0.000016,0.0,0.0,0.0,0.0


데이터 전처리

In [111]:
def preprocessing(data):
  '''
  feature데이터를 입력하여, 모델 입력용 데이터로 변경
  '''
  # 충돌체 별로 가속도 데이터와 시간 데이터를 id로 묶음
  _data = data.groupby('id')

  # string 형태로 반환
  _data['Time']  = _data['Time'].astype('str')

  # 모델에 입력이 가능한 1차원 형태로 변환
  _data = _data.pivot_table(index = 'id', columns = 'Time', values = ['S1', 'S2', 'S3', 'S4'])

  # col명 변경
  _data.columns =['_'.join(col) for col in _data.columns.values]

  return _data 

In [112]:
train_features = preprocessing(train_features)
test_features = preprocessing(test_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [113]:
train_features.head()

,S1_0.0,S1_0.0001,S1_0.00010400000000000001,S1_0.00010800000000000001,S1_0.00011200000000000001,S1_0.00011599999999999999,S1_1.2e-05,S1_1.6e-05,S1_2.4e-05,S1_2.8000000000000003e-05,S1_2e-05,S1_3.2e-05,S1_3.6e-05,S1_4.4e-05,S1_4.8e-05,S1_4e-05,S1_4e-06,S1_5.2000000000000004e-05,S1_5.6000000000000006e-05,S1_5.9999999999999995e-05,S1_6.4e-05,S1_6.8e-05,S1_7.2e-05,S1_7.6e-05,S1_8.4e-05,S1_8.8e-05,S1_8e-05,S1_8e-06,S1_9.2e-05,S1_9.6e-05,S2_0.0,S2_0.0001,S2_0.00010400000000000001,S2_0.00010800000000000001,S2_0.00011200000000000001,S2_0.00011599999999999999,S2_1.2e-05,S2_1.6e-05,S2_2.4e-05,S2_2.8000000000000003e-05,...,S3_6.4e-05,S3_6.8e-05,S3_7.2e-05,S3_7.6e-05,S3_8.4e-05,S3_8.8e-05,S3_8e-05,S3_8e-06,S3_9.2e-05,S3_9.6e-05,S4_0.0,S4_0.0001,S4_0.00010400000000000001,S4_0.00010800000000000001,S4_0.00011200000000000001,S4_0.00011599999999999999,S4_1.2e-05,S4_1.6e-05,S4_2.4e-05,S4_2.8000000000000003e-05,S4_2e-05,S4_3.2e-05,S4_3.6e-05,S4_4.4e-05,S4_4.8e-05,S4_4e-05,S4_4e-06,S4_5.2000000000000004e-05,S4_5.6000000000000006e-05,S4_5.9999999999999995e-05,S4_6.4e-05,S4_6.8e-05,S4_7.2e-05,S4_7.6e-05,S4_8.4e-05,S4_8.8e-05,S4_8e-05,S4_8e-06,S4_9.2e-05,S4_9.6e-05
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,28.861840,34.469910,2.820700e+01,-17.249630,-113.524700,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000,-0.000001,-0.000008,-4.972607e-08,0.0,-0.000067,-0.000369,-0.00159,-0.004151,-0.003038,0.044299,2.704033e-01,3.079798,6.975893,1.057531,0.0,13.266300,20.672990,0.0,28.861840,34.469910,28.207000,-17.249630,-113.524700,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000,0.0,0.000111,0.000561,0.002209,0.008652,0.029441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,3.429466e-07,0.000000,0.0,0.000003,0.000023
1,0.0,0.000000,0.000000,3.147750e-07,0.000002,0.000009,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,13210.550000,15362.070000,14.563430,-18139.340000,-25793.510000,0.0,0.0,-0.000021,-0.003461,...,0.0,0.0,1.053139e-06,6.347580e-06,0.000221,1.232694e-03,4.514454e-05,0.0,5.128649e-03,0.022619,0.0,64.223520,93.032010,121.873300,134.589200,75.043920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00000,1.446068e-06,0.000040,0.000571,4.526681e-03,0.031795,0.155535,2.419442,7.261036e+00,0.693200,0.0,18.180060,37.053800
2,0.0,-25719.010000,-36065.610000,-2.211723e+04,19586.620000,75466.530000,0.0,-0.000092,-0.155149,-3.675771,-6.843146e-03,-25.13809,-99.968380,126.648400,1067.813000,-1.874429e+02,0.0,1165.781000,-1203.031000,-3605.58200,-1432.085000,4507.028000,5000.117000,-1.202514e+03,-2002.576000,523.418500,-4422.679000,0.0,-32.091780,-8347.370000,0.0,0.000023,0.000124,0.000513,0.002150,0.007754,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,2.869836e-08,0.000000e+00,0.0,1.545781e-07,0.000001,0.0,0.241764,0.683214,1.693787,3.765712,7.470505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00000,0.000000e+00,0.000000,0.000000,2.423671e-07,0.000005,0.000032,0.001136,5.646074e-03,0.000236,0.0,0.021411,0.078795
3,0.0,0.002897,0.012184,4.202466e-02,0.138205,0.400494,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-7.978055e-08,0.000002,0.000021,0.000000,0.0,0.000118,0.000694,0.0,-1733.572000,-197.595900,2284.228000,1979.613000,-954.100600,0.0,0.0,0.000000,0.000000,...,0.0,0.0,-8.178223e-08,-2.571635e-07,0.000008,8.358666e-05,5.255491e-07,0.0,4.305689e-04,0.002314,0.0,26.631090,35.400350,41.752940,21.546210,-54.884090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.062450e-07,0.0,0.0,-0.00001,-7.746829e-05,-0.000452,-0.001540,-2.029647e-03,0.018408,

In [119]:
train_target.head()

,id,X,Y,M,V
0,0,0.0,-400.0,50.0,0.4
1,1,400.0,0.0,100.0,1.0
2,2,-300.0,-200.0,25.0,0.4
3,3,200.0,-100.0,150.0,0.4
4,4,-300.0,-100.0,150.0,0.4


In [114]:
test_features.shape

(700, 120)

변수 선택 및 모델 구축

In [ ]:
# ensenble (rf)

In [115]:
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [120]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

rf1 = MultiOutputRegressor(RandomForestRegressor(random_state=42, bootstrap=True, max_features='sqrt'), n_jobs=-1)

param_dstn_rf1 = {
    'estimator__n_estimators': list(range(100, 1001, 25)),
    'estimator__max_depth': list(range(5, 51, 3)),
    'estimator__min_samples_split': list(range(5, 51, 3)),
    'estimator__min_samples_leaf': list(range(5, 51, 3)),
    'estimator__max_leaf_nodes': list(range(5, 51, 3))
}

# 아무리 생각해도 param_dstn_rf1 이거 범위를 잘못 준 거 같음

random_rf1 = RandomizedSearchCV(estimator=rf1,
                                param_distributions=param_dstn_rf1,
                                n_iter=1000,
                                n_jobs=-1,
                                cv=5,
                                random_state=42,
                                scoring='neg_mean_squared_error',
                                verbose=2)

In [ ]:
y_pred = random_rf1.fit(train_features, train_target)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 34.0min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 78.6min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 138.3min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 216.6min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 296.4min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 408.6min


In [ ]:
predict1 = random_rf1.best_estimator_.predict(test_features)

In [124]:
rf2 = RandomForestRegressor(random_state=42)

param_dstn_rf2 = {
    'n_estimators': list(range(50, 100, 20)),
    'max_depth': list(range(5, 11)),
    'min_samples_split': list(range(2, 15)),
    'min_samples_leaf': list(range(2, 11))
}

random_rf2 = RandomizedSearchCV(estimator=rf2,
                                param_distributions=param_dstn_rf2,
                                n_iter=100,
                                n_jobs=-1,
                                cv=3,
                                random_state=42,
                                scoring='neg_mean_squared_error',
                                verbose=1)

In [129]:
predict2 = random_rf2.fit(train_features, predict1)

TypeError: ignored

In [22]:
# 제출 형태 
submit = pd.read_csv("sample_submission.csv")
submit.head()

,id,X,Y,M,V
0,2800,0,0,0,0
1,2801,0,0,0,0
2,2802,0,0,0,0
3,2803,0,0,0,0
4,2804,0,0,0,0


In [23]:
# 예측한 값 적기
for i in range(4):
  submit.iloc[:,i+1] = predict2[:,i]

In [24]:
submit.head()

,id,X,Y,M,V
0,2800,1411.71,-261.0,-39.0,86.25
1,2801,1535.97,276.0,-307.0,91.75
2,2802,1465.22,-253.0,71.0,96.25
3,2803,1175.57,151.0,158.0,113.50
4,2804,1369.44,-207.0,219.0,102.00


In [25]:
# 저장 
submit.to_csv('baseline_test2.csv',index = False)

In [ ]:
# cnn은 시도2에서 시도해봄! 